# Language Model
This time we'll use Markov models to generate text rather than classify

so unsupervised model

- Classifiying text: labels exist (supervised learning)
- Generating text: no labels (unsupervised learning)

Burda anlamadığım bi slayt oldu, 39.derste.

### Sampling (more details in ipad)
- suppose our vocab is "cat", "dog", "mouse" with probabilities 0.2, 0.5, 0.3
- sampling from this vocab will be: if we idx the words to integers 1,2,3 (yoksa 0,1,2 mi muhtemelen öyle)

In [1]:
np.random.choice(3, p = [0.2, 0.5, 0.3])

NameError: name 'np' is not defined

### Problems with Markov Assumption and Extending the Markov model
- only limited to single previous word
- what if depend on two past state:
First Order Markov : p(st|st-1, st-2...) = p(st|st-1)
Second Order Markov: p(st|st-1, st-2...) = p(st|st-1, st-2)

#### Implementation:
Aijk = p(st = k | st-1 = j, st-2 = i)
- so the first order markov model is a 3d array (a tensor)
- the shape is MxMxM M-cube
- fourth order: M^3 , fifth order: M^4

#### The full model:
- πi = p(s1 = i)
- A1ij = p(s = j | s1 = i)
- A2ijk = p(st = k | st-1 = j, t-2 = i)

#### Be careful A1 is no longer a state transition matrix for all transition, it's only used for transitionin from first to second word not any word after that

#### On the otherhand in Deep Learning, no need for Markov assumption, no computation cost for adding extra previous words, well, it depends on the NN used, but think it as a linear increase instead of exponential increase in computation cost
ex: modeling (st|st-1, st-2, st-3.., st-T+1) would be O(M^T) in Markov Model but O(T) in neural network

Exercise: how is this possible? What is NN actually doing? 
- hmm flattening before the computation ?

## Exercise Prompt

- download data : poems by Frost
- build train markov model, we'll not use matrices arrays but dicitonaries? 
- use the model to generate poems, we'll generate four lines at atime
#### note this exercise is very open-ended, o yüzden storing için dictionary kullancağımızı söyledi
- ex: you're working on initial word distribution
π = {"the":0.2, "a" : 0.3, "is": 0.1, ...}
key is the word and value is the prob
- note we'll not use add-one smoothing so the number of key-value pairs in the dictionary will be the number of words used in the first line of poem,  many words will have zero probability (not appear in the dictionary)
- although there are V words in the vocab the dictionary will not store them
- this is opposed to the π vector, which has always size V
#### then why dictionaries instead of np arrays
- Sparsity!
- remember many transitions get zero as they don't appear in the corpus
- this get worse and worse as the model order get bigger
- altho number of words grow exponentially with the order, the amount of data we actually get gets smaller in comparison
- our transition arrays will mostly be zeores out of VxVxV elements!!!
- so it's more efficient to store transitions we see

### First Order Transitions
recall: used only to model the 2nd word in line
- we'll use a dictionary of dicionaries
A1 = {"the" : 
{"cat": 0.02, "dog": 0.01...},
"a": {...}
...
}
- exercise how you store second order transitions using a similar scheme? #ay çok zor

### Sampling from a probability dictionary
- how do we sample words from a dictionary with probabilities stored as values and words as keys? 
- karışık bi yöntem açıkladı cumulative sum ve boundary üzerinden
- alternatively convert keys and values into lists then use np.random.choice() but then you don2t improve a critical skill

In [1]:
import numpy as np
import string #bu neye yarıyo acep

np.random.seed(1234)

In [2]:
#ınitialize the dictionaries: 
initial = {} # to store the distribution of start of a phrase 
first_order = {} # store the transition prob only for the sec word
second_order = {} #store transition prob for the rest

In [3]:

def remove_punctuation(s):
    return s.translate(str.maketrans('','',string.punctuation))

In [4]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

File 'robert_frost.txt' already there; not retrieving.



In [5]:
def add2dict(d, k, v): #dictionary, key and value as inputs
    #k will represent a starting word, or a pair of word in the second order case
    if k not in d: #dictionary'de değilse dictionary'ye ekle
        d[k] = [] 
    d[k].append(v) #niye append kullandık da eşitlemedik ? #çünkü başka deperler de alabilsin
    
# [cat, cat, dog, dog, dog, dog, dog, mouse, ...]
#ANLAMADIM

#sonraki slaytta anlatmış: 
#value of this dictionary will be a list

 - I am happy
 - I am sad
 - I am content
 - I am happy
 - I am happy
 - I am hungry
 
 #### Intended outcome:
 
{("I","am"): ["happy", "sad", "content", "happy","happy","hungry"]}

#### Later:
{("I","am"): {"happy":0.5, ...}} bu yüzden append

In [6]:
##populate the dictionaries:
for line in open('robert_frost.txt'):
    tokens = remove_punctuation(line.rstrip().lower()).split()
    
    T = len(tokens)
    for i in range(T):
        t = tokens[i]
        if i == 0:
            initial[t] = initial.get(t,0.) + 1 #get kısmında olanı anlamadım,
            #if t does not exist in the dictionary yet, it automaticaly gets a count of zero with this code
        else:
            t_1 = tokens[i-1]
            if i == T-1: #yani son kelimeyse
                #measure probability of ending the line
                add2dict(second_order, (t_1,t), "END") 
                #second_order isimli dictionaryde (t_1, t) adlı bi key yarat, onun listesine "END"'ekle (append)
            if i == 1: #yani ikinci kelime
                ###NOTE THIS SECOND IF IS A NEW STATEMENT TO RUN ON THE SAME TOKEN, NOT USE ELIF
                ### BECAUSE THESE TWO CONDTIONS NOT MUTUALLY EXCLUSIVE T-1 == 1 OLABİLİR
                add2dict(first_order, t_1, t) ##bunu kendim buldum
            else:
                t_2 = tokens[i-2]
                add2dict(second_order,(t_2, t_1), t) #bunu da kendim buldum

In [7]:
initial

{'two': 8.0,
 'and': 129.0,
 'to': 50.0,
 'then': 12.0,
 'because': 1.0,
 'though': 7.0,
 'had': 4.0,
 'in': 29.0,
 'oh': 4.0,
 'yet': 3.0,
 'i': 118.0,
 'somewhere': 1.0,
 'whose': 2.0,
 'his': 7.0,
 'he': 34.0,
 'my': 7.0,
 'between': 3.0,
 'the': 82.0,
 'of': 29.0,
 'but': 51.0,
 'some': 5.0,
 'from': 10.0,
 'is': 5.0,
 'natures': 1.0,
 'her': 2.0,
 'so': 13.0,
 'nothing': 2.0,
 'when': 9.0,
 'came': 1.0,
 'one': 11.0,
 'proclaimed': 1.0,
 'smoothlaid': 1.0,
 'half': 3.0,
 'up': 7.0,
 'a': 30.0,
 'disturbed': 2.0,
 'comes': 1.0,
 'by': 11.0,
 'if': 12.0,
 'were': 3.0,
 'ten': 1.0,
 'as': 14.0,
 'it': 23.0,
 'lifted': 1.0,
 'she': 17.0,
 'transfixed': 1.0,
 'across': 1.0,
 'was': 9.0,
 'once': 2.0,
 'that': 29.0,
 'looked': 1.0,
 'stood': 1.0,
 'you': 40.0,
 'me': 1.0,
 'sometimes': 4.0,
 'brown': 2.0,
 'cross': 1.0,
 'describing': 1.0,
 'got': 1.0,
 'walls': 1.0,
 'like': 8.0,
 'upon': 2.0,
 'with': 18.0,
 'faster': 1.0,
 'sitting': 1.0,
 'according': 1.0,
 'hes': 11.0,
 'or': 11.0,

In [8]:
first_order

{'two': ['roads',
  'roads',
  'miles',
  'oldbelievers',
  'winds',
  'weeks',
  'of',
  'at'],
 'and': ['sorry',
  'be',
  'looked',
  'having',
  'both',
  'that',
  'miles',
  'miles',
  'would',
  'dropped',
  'further',
  'when',
  'tell',
  'the',
  'caught',
  'put',
  'threw',
  'birds',
  'suddenly',
  'scurf',
  'sorry',
  'since',
  'whats',
  'tell',
  'many',
  'blew',
  'stamped',
  'sometimes',
  'some',
  'then',
  'came',
  'this',
  'then',
  'politician',
  'thatd',
  'rode',
  'if',
  'from',
  'i',
  'he',
  'full',
  'experts',
  'built',
  'both',
  'thats',
  'spoke',
  'anyway',
  'had',
  'the',
  'how',
  'taken',
  'lie',
  'left',
  'stroked',
  'the',
  'a',
  'me',
  'a',
  'between',
  'wont',
  'hes',
  'his',
  'nothing',
  'better',
  'kick',
  'carried',
  'thought',
  'swollen',
  'swollen',
  'hold',
  'all',
  'fell',
  'set',
  'sit',
  'bring',
  'push',
  'that',
  'those',
  'sproutlands',
  'perhaps',
  'see',
  'dangle',
  'disappeared',
  

In [9]:
second_order

{('two', 'roads'): ['diverged', 'diverged'],
 ('roads', 'diverged'): ['in', 'in'],
 ('diverged', 'in'): ['a', 'a'],
 ('in', 'a'): ['yellow',
  'wood',
  'window',
  'packing',
  'byroad',
  'family',
  'new',
  'row',
  'time',
  'town',
  'book',
  'smother',
  'glass',
  'glass'],
 ('yellow', 'wood'): ['END'],
 ('a', 'yellow'): ['wood'],
 ('and', 'sorry'): ['i', 'i'],
 ('sorry', 'i'): ['could', 'ever'],
 ('i', 'could'): ['not',
  'END',
  'have',
  'see',
  'see',
  'do',
  'see',
  'have',
  'not',
  'END'],
 ('could', 'not'): ['travel', 'say'],
 ('travel', 'both'): ['END'],
 ('not', 'travel'): ['both'],
 ('and', 'be'): ['one', 'whole'],
 ('be', 'one'): ['traveler'],
 ('one', 'traveler'): ['long'],
 ('traveler', 'long'): ['i'],
 ('i', 'stood'): ['END'],
 ('long', 'i'): ['stood', 'lay'],
 ('and', 'looked'): ['down'],
 ('looked', 'down'): ['one', 'the'],
 ('down', 'one'): ['as', 'street'],
 ('one', 'as'): ['far'],
 ('as', 'far'): ['as', 'as'],
 ('far', 'as'): ['i', 'we'],
 ('as', 'i')

In [10]:
## normalize the initial distribution since this is for now the only dictionary with counts
initial_total = sum(initial.values())
for t,c in initial.items(): ### items'i unutma
    initial[t] = c / initial_total

In [11]:
initial

{'two': 0.005571030640668524,
 'and': 0.08983286908077995,
 'to': 0.034818941504178275,
 'then': 0.008356545961002786,
 'because': 0.0006963788300835655,
 'though': 0.004874651810584958,
 'had': 0.002785515320334262,
 'in': 0.0201949860724234,
 'oh': 0.002785515320334262,
 'yet': 0.0020891364902506965,
 'i': 0.08217270194986072,
 'somewhere': 0.0006963788300835655,
 'whose': 0.001392757660167131,
 'his': 0.004874651810584958,
 'he': 0.023676880222841225,
 'my': 0.004874651810584958,
 'between': 0.0020891364902506965,
 'the': 0.057103064066852366,
 'of': 0.0201949860724234,
 'but': 0.035515320334261836,
 'some': 0.003481894150417827,
 'from': 0.006963788300835654,
 'is': 0.003481894150417827,
 'natures': 0.0006963788300835655,
 'her': 0.001392757660167131,
 'so': 0.009052924791086351,
 'nothing': 0.001392757660167131,
 'when': 0.006267409470752089,
 'came': 0.0006963788300835655,
 'one': 0.00766016713091922,
 'proclaimed': 0.0006963788300835655,
 'smoothlaid': 0.0006963788300835655,
 'h

In [12]:
# convert [cat, cat, cat, dog, dog, dog, dog, mouse, ...]
# into {cat: 0.5, dog: 0.4, mouse: 0.1}

#this step will do two thngs: 1- create a dict of counts 2-normalize the counts convert to probs

def list2pdict(ts): #ts is a list ot tokens
    d = {}
    n = len(ts)
    for t in ts:
        d[t] = d.get(t,0.) + 1
    for t, c in d.items():
        d[t] = c / n # n yani len(ts) zaten sum of dict values'e eşit, toplamadan önce hepsinin yan yana yazılmış haliydi
        
    return d        

In [13]:
#yalnız burdaki d tranformation'ı sadece tek listeye uygulanmış olmuyo mu, nasıl tüm listelre uygulıycaz? 

#BÖYLE
for t_1, ts in first_order.items():
  # replace list with dictionary of probabilities
  first_order[t_1] = list2pdict(ts)

In [14]:
first_order

{'two': {'roads': 0.25,
  'miles': 0.125,
  'oldbelievers': 0.125,
  'winds': 0.125,
  'weeks': 0.125,
  'of': 0.125,
  'at': 0.125},
 'and': {'sorry': 0.015503875968992248,
  'be': 0.007751937984496124,
  'looked': 0.007751937984496124,
  'having': 0.007751937984496124,
  'both': 0.015503875968992248,
  'that': 0.015503875968992248,
  'miles': 0.015503875968992248,
  'would': 0.007751937984496124,
  'dropped': 0.007751937984496124,
  'further': 0.007751937984496124,
  'when': 0.007751937984496124,
  'tell': 0.023255813953488372,
  'the': 0.031007751937984496,
  'caught': 0.007751937984496124,
  'put': 0.015503875968992248,
  'threw': 0.007751937984496124,
  'birds': 0.007751937984496124,
  'suddenly': 0.007751937984496124,
  'scurf': 0.007751937984496124,
  'since': 0.015503875968992248,
  'whats': 0.007751937984496124,
  'many': 0.007751937984496124,
  'blew': 0.007751937984496124,
  'stamped': 0.007751937984496124,
  'sometimes': 0.007751937984496124,
  'some': 0.007751937984496124,

In [15]:
for t_2_1, ts in second_order.items():
    second_order[t_2_1] = list2pdict(ts)
second_order

{('two', 'roads'): {'diverged': 1.0},
 ('roads', 'diverged'): {'in': 1.0},
 ('diverged', 'in'): {'a': 1.0},
 ('in', 'a'): {'yellow': 0.07142857142857142,
  'wood': 0.07142857142857142,
  'window': 0.07142857142857142,
  'packing': 0.07142857142857142,
  'byroad': 0.07142857142857142,
  'family': 0.07142857142857142,
  'new': 0.07142857142857142,
  'row': 0.07142857142857142,
  'time': 0.07142857142857142,
  'town': 0.07142857142857142,
  'book': 0.07142857142857142,
  'smother': 0.07142857142857142,
  'glass': 0.14285714285714285},
 ('yellow', 'wood'): {'END': 1.0},
 ('a', 'yellow'): {'wood': 1.0},
 ('and', 'sorry'): {'i': 1.0},
 ('sorry', 'i'): {'could': 0.5, 'ever': 0.5},
 ('i', 'could'): {'not': 0.2, 'END': 0.2, 'have': 0.2, 'see': 0.3, 'do': 0.1},
 ('could', 'not'): {'travel': 0.5, 'say': 0.5},
 ('travel', 'both'): {'END': 1.0},
 ('not', 'travel'): {'both': 1.0},
 ('and', 'be'): {'one': 0.5, 'whole': 0.5},
 ('be', 'one'): {'traveler': 1.0},
 ('one', 'traveler'): {'long': 1.0},
 ('t

In [16]:
## create a function that will sample a word, given the list of probabilities

def sample_word(d):
    p0 = np.random.random() #first draw a sample from a uniform distribution, a number btw zero and 1
    cumulative = 0 #we'll store the cumulative sum of probabilites we encounter in dict
    for t,p in d.items():
        cumulative += p ## böyle böyle son t,p'de cumulative sum 1'e ulaşcak
        if p0 < cumulative : #eğer o random value bu kümülatiften küçükse t bölgesine düşer, büyükse ??
            #e bu loop sonsuza kadar devam ederse t'den sonraki değerlerin hepsini de verir, bu loop'u durdurmuyo muyuz
            return t
    assert(False) #should never get here: as a sanity check that our function should never reach this line
    #if reaches here program will raise an exception & we'll know done some thing wrong

In [17]:
def generate():
    for i in range(4): # generate 4 lines
        sentence = []

    # initial word
        w0 = sample_word(initial)
        sentence.append(w0)

    # sample second word
        w1 = sample_word(first_order[w0])
        sentence.append(w1)

    # second-order transitions until END
        while True: #infinite loop
            w2 = sample_word(second_order[(w0, w1)]) #recall keys to this dictionary are tuples
            if w2 == 'END': #exit condition from inf loop
                break #without appending anything
            sentence.append(w2) #otherwise append the final word
            w0 = w1
            w1 = w2
        print(' '.join(sentence))


In [22]:
generate()

first theres the childrens house of makebelieve
are there they said it and besides
and this bill
my french indian esquimaux


### Exercise:

- Determine the vocabulary size (V)
- We know that pi has shape V, A1 has shape V x V, and A2 has shape V x V x V
- In comparison, how many values are stored in our dictionaries?

### Exercise 2:
- We can skip the step where we accumulate all the possible next words in a listi E.g. [cat, cat, dog, dog, dog, ...]
- Instead, like we do with the initial state distribution, create the dictionary of counts directly as you loop through the data.
- You'll no longer need list2pdict()
- modify this code so you'll no longer need to collect each possible next word in a list.

#### One advantage of dictionnaries since key to dictionaries are any type of object, you don't need to convert words to integers and move back and forth, one of the most tedious part of nlp